# Introduction

    For this project, I have used topic modeling to deliver marketing and product insights for SPEEDO.

# The Data

    Prof. Julian McAuley at UC-San Diego has graciously let me use his “Amazon Product Data. It contains tons of data about Amazon products. Specifically, I have leveraged two datasets: (1) meta-data about products & (2) product reviews. The aforementioned database has reviews on all types of Amazon products,but these datasets are huge (~80gb).

    To remedy this, I’ve picked to two smaller datasets that only contain (1) meta-data about products that are in    categorized as “Clothing, Shoes & Jewelry” and (2) reviews about products that are in the “Clothing, Shoes &    Jewelry” category.

# Preprocessing Steps:
    Firstly, I extracted the Asins(unique identifier) for all products of SPEEDO from the 
    Metadata of products.

In [1]:
count = 0 
loadedjson = open('meta_Clothing_Shoes_and_Jewelry.json','r')

# loading the json metadata of products 

In [2]:
allproducts = {}    #making a dictionary
listofcategories = {}

for aline in loadedjson:
    count += 1
    if count % 100000 == 0:
        print(count)
    aproduct = eval(aline)
    
    allproducts[aproduct['asin']] = aproduct
    
    for categories in aproduct['categories']:
        for acategory in categories:
            if acategory in listofcategories:
                listofcategories[acategory] += 1
            if acategory not in listofcategories:
                listofcategories[acategory] = 1

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [3]:
count = 0 
allspeedoasins = set()

for aproduct in allproducts:
     theproduct = allproducts[aproduct]
     count += 1
     if count % 100000 == 0:
         print(count/1503384)
     for categories in theproduct['categories']:
        for acategory in categories:
            if 'speedo' in acategory.lower():
                allspeedoasins.add(theproduct['asin'])    

0.06651660520532346
0.13303321041064692
0.19954981561597038
0.26606642082129384
0.33258302602661727
0.39909963123194075
0.4656162364372642
0.5321328416425877
0.5986494468479111
0.6651660520532345
0.7316826572585581
0.7981992624638815
0.8647158676692049
0.9312324728745284
0.9977490780798518


In [4]:
print ("length of all products in the data is: ")
print(len(allproducts))

length of all products in the data is: 
1503384


In [5]:
print ("length of Speedo products is: ")
print(len(allspeedoasins))

length of Speedo products is: 
1625


In [6]:
outputfile = open('allasins.txt', 'w')
# Opening a text file 

outputfile.write(','.join(allspeedoasins))
#overwriting the text file by joining allspeedoasins 

outputfile.close()
# Closing the file 

# Step 2:
    Secondly, I created a dictionary of all Speedo product reviews then extracted the reviews from review data and    dumped them into a json file.

In [7]:
loadedjson = open('reviews_Clothing_Shoes_and_Jewelry.json','r')
# loading json data of reviews

In [8]:
allreviews = {}
count = 0

for aline in loadedjson:
    count += 1
    if count % 100000 == 0:
        print(count)
    areview = eval(aline)
    
    theasin = areview['asin']
    thereviewer = areview['reviewerID']
    
    if theasin in allspeedoasins:
        thekey = '%s,%s'%(theasin,thereviewer)
        allreviews[thekey] = areview

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000


In [9]:
print ("Length of reviews of Speedo is: ")
len(allreviews)

Length of reviews of Speedo is: 


9573

In [10]:
import json
        
json.dump(allreviews,open('allspeedoreviews.json','w'))  # allspeedoreviews

allreviews = json.load(open('allspeedoreviews.json','r'))
# This code will open the json file created above 

# Step 3:
    In the next step I have removed the stop-words like “a”, “an” as these words are so common  in English sentence that they can be over represented in the results. For this, I have imported nltk(natural language toolkit).

In [11]:
import nltk    
# removing stopwords from the reviews here to fetch better topic terms

nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

stop_words = stopwords.words('english')

stop_words.append('speedo')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adititripathi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 4:
    Here I have defined a function that would load the asins, summary and reviewtext together in a set for further    processing.

In [16]:
texts = set()

def load_texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            print(topicdata[areview].keys())
            reviewtext = topicdata[areview]['reviewText']
            summary = topicdata[areview]['summary']
            asin = topicdata[areview]['asin']
            review = '%s %s %s' %(asin,summary,reviewtext)
            
            texts.add(review)

#load_texts(allreviews)

# The model I chose for this project is K-means clustering.

    At first I have imported TfidVectorizer from sklearn feature extraction and K means from sklearn cluster.
    K – means initializes with a pre-determined number of clusters which is 22 in my model.
    The maximum iteration which I set in my model is 100000 i.e my model would only iterate the process until 
    100000 times.
    The next step involves creating an organized list of topics.
    After having the topics, I have create a directory for classification of topic terms thereby extracting the 
    topic terms from each topic.
    A dictionary entry is created for each topic which is initially empty. The topic terms are used to name files  and at the end of the loop the file is closed to make sure that nothing could be read & written further in it.


# Model 1

    Basic Model with no additional conditions.

In [15]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   
# Assigning number of topics, it can be changed later analysing the overlaps among topics.  

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))
    

Top terms per cluster
0: stars five four three
1: pool shoes feet great
2: tight size like top
3: watch band lap time
4: good quality fit great
5: small size runs suit
6: trunks swim great fit
7: bag b000791api gear equipment
8: nice fit color good
9: shirt b005qxesww sun great
10: size ordered big smaller
11: great fit one well
12: shorts great swim board
13: 34 size fit 32
14: comfortable great shoes beach
15: old daughter year loves
16: shoe water great shoes
17: swimsuit great fit daughter
18: shoes water great b0010odjnk
19: product great good quality
20: suit great bathing swim
21: flip flops flop comfortable


In [17]:
import os
outfiles = {}

try:
    os.mkdir('output')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    # it extracts the topic terms from each topic term
    outfiles[atopic] = open(os.path.join('output', '_'.join(topic_terms) + '.txt'), 'w')  
    #creating a dictionary entry for a topic, and I am taking those 
    #topic terms and using them to name the files

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        #predictions = model.predict(Y)
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            #here I am making sure by piping in,that I am adding a line break


for n, f in outfiles.items():
    f.close() 

directory already exists


# Most relevant topic terms and their descriptions.

    1: trunks swim fit great - Wide range of Men's shorts are comfortable, fit well & holds color.
    3: aerobics water class shoes -  Water shoes by Speedo which are perfect for water aerobics.
    5: shirt b005qxesww sun great - Speedo rash guard t-shirt provides great fit, sun protection & comes in 
    good quality nylon fabric.
    19: watch band lap time - Speedo Aquacoach watch has short strap opposed to size mentioned on site. It measures every stroke distance & time accurately.
    21: bag b000791api equipment great -  The mesh bag is big enough to store plenty of stuff.

# Model 2

    Negative Reviews - starred less than 2 in 2013   

In [18]:
texts = set()

def load2texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) < 2:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2013":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

#load2texts(allreviews)

In [19]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: wet pool day got
1: shoe water shoes return
2: came stitching time first
3: suit b0096s1592 modest back
4: use months started item
5: apart fell months shoes
6: watch time broke counter
7: flip plastic flops price
8: material fit size difficult
9: shoes feet water never
10: small size way 34
11: tight look picture like
12: sizing size shirt small
13: shorts told vietnam pockets
14: faded fabric color one
15: wet shoes well slipped
16: suit poor quality fabric
17: arch makes uncomfortable something
18: thin toe unglued comes
19: quality bad good buy
20: heel money throw hated
21: seller product warranty worn


In [20]:
# Classification of topic terms

import os
outfiles = {}

try:
    os.mkdir('output2')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output2', '_'.join(topic_terms) + '.txt'), 'w')  
    
for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
       
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
           


for n, f in outfiles.items():
    f.close()     

directory already exists


# Most relevant topic terms and their descriptions.

    2: wet beware dangerous hurt - Speedo shoes are quite comfortable but slip on wet floors.
    12: sizing chart misleading small - Fabric is stretchy and comfortable, but sizing is off!
    14: narrow brand inch side -  The side edges of brief are too narrow: highly disappointing.
    15: watch buy counter return -  Watch is good but band is ugly.
    21: flip flops toes plastic - The strap is made up of horrible plastic and cause blisters.

# Model 3

    Positive Reviews - starred more than 4 in 2013  

In [27]:
texts = set()

def load3texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) > 4:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2013":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load3texts(allreviews)

In [28]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: beach great perfect comfortable
1: good quality price nice
2: suit great swim fits
3: shoes water great b0010odjnk
4: swimsuit like suits swimming
5: runs size small large
6: love great comfortable fit
7: b001ibiwqy comfortable great sandal
8: trunks swim fit great
9: shoe water great comfortable
10: suit bathing fits daughter
11: one big size fit
12: shorts swim great board
13: 34 fit great fits
14: size order one larger
15: fit perfect size ordered
16: pool aerobics water great
17: old year loves daughter
18: bag b000791api gear swim
19: product great expected fit
20: great water shoes comfortable
21: shirt sun protection b005qxet1m


In [29]:
# Classification of topic terms

import os
outfiles = {}

try:
    os.mkdir('output3')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output3', '_'.join(topic_terms) + '.txt'), 'w') 
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
           


for n, f in outfiles.items():
    f.close()  

directory already exists


# Most relevant topic terms and their descriptions.

    0: brand chose swimmer perfect -   Recommended for everyday swimmer.
    1: great price value fit -  SPEEDO WOMEN'S SURFWALKER PRO - A great product. Costed a few more dollars, but 
    well worth the price.
    4: quality nice good little -  Speedo Men's Solid Jammer The material is great and very sleek design.
    7: shoes water great comfortable -  GREAT FOR NON-SLIP! These shoes are perfect for cave tubing.
    13: old year loves daughter -   Baby girl products are adorable and secure.
    

# Model 4

    Negative Reviews - starred less than 2 in 2014 

In [30]:
texts = set()

def load4texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) < 2:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2014":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load4texts(allreviews)

In [31]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: worked lap minutes got
1: small waist fit like
2: shoes return small water
3: shoe give disappointed size
4: short shorts b0047e59lu 8217
5: apart strap b001ibiwqy foot
6: size small ordered large
7: stopped working received days
8: suit lining bottom compression
9: watch water horrible died
10: flops flip 34 feet
11: buttons even instructions directions
12: see good soon b004477t18
13: quality poor pair years
14: big color back send
15: inside seams never blisters
16: 34 black wear ya
17: waterproof one news recomend
18: water well time first
19: bag hard making complaint
20: skin b006pkj13g small son
21: broken sent replacement use


In [32]:
import os
outfiles = {}

try:
    os.mkdir('output4')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output4', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()     
               

directory already exists


# Most relevant topic terms and their descriptions.

    2: pair apart poor quality - Non-Standard sizes and dense tight fabric – Speedo jammers
    7: watch water horrible using - Speedo watches have technical issues, stops working in few days of use.
    20: tight strange fit boring - Inappropriate range of size – XL is not a regular XL.
    21: strap apart b001ibiwqy came -  Disaster Strap! Men’s pool slide.
    

# Model 5

    Positive Reviews - starred more than 4 in 2014 

In [33]:
texts = set()

def load5texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) > 4:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2014":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load5texts(allreviews)

In [34]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: jammers b0006uzx9m fit going
1: 34 fit great perfect
2: pool b000c3vb1m great foot
3: suit great bathing fits
4: tight little like size
5: easy water comfortable b0010odjnk
6: love fit perfectly b0010odjnk
7: comfortable great fit perfect
8: well made fit great
9: five stars great good
10: loves daughter suit swim
11: shoe water great fit
12: size small great 10
13: long love sun shorts
14: shoes water b0010odjnk great
15: shorts swim great fit
16: bag b000791api gear great
17: shirt b005qxesww great nice
18: old year suit daughter
19: one size larger suit
20: good quality product great
21: cute fit great wear


In [35]:
import os
outfiles = {}

try:
    os.mkdir('output5')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output5', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()  

directory already exists


# Most relevant topic terms and their descriptions.

    3: good quality fit well - Great result: for wide variety of footwear.
    13: excellent beach walk rocks - Great for comfortable beachcoming: Speedo Surfwalkers
    14: suit great bathing fits -  A wide variety of Men’s swimsuit tend to fit comfortably.
    21: looking well great made -  Brief swimsuit: snug fitting.
 

# Model 6

    Reviews during summer 2013(June - August)

In [36]:
texts = set()

def load6texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            reviewdate = topicdata[areview]['reviewTime']
            datenocomma = reviewdate.replace(",", "")
            mmddyy = datenocomma.split(' ')
            if (int(mmddyy[0])>5 and int(mmddyy[0])<9):
                summary = topicdata[areview]['summary']
                reviewtext = topicdata[areview]['reviewText']
                asin = topicdata[areview]['asin']
                review = '%s %s %s' %(asin,summary,reviewtext)
                texts.add(review)

load6texts(allreviews)

In [37]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: suit great bathing fits
1: top one great like
2: swimsuit great daughter fits
3: good fit well fits
4: size small ordered tight
5: watch band lap laps
6: beach sand shoes great
7: pool pair comfortable great
8: shoes feet water pair
9: stars four three two
10: shirt great b005qxesww sun
11: five stars great good
12: large size big medium
13: perfect fit daughter loves
14: shoes water great b0010odjnk
15: quality good size nice
16: shoe water size great
17: b002mukiz8 comfortable wear water
18: product great good quality
19: bag b000791api gear mesh
20: shorts swim great short
21: 34 size fit waist


In [38]:
import os
outfiles = {}

try:
    os.mkdir('output6')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output6', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()   

directory already exists


# Most relevant topic terms and their descriptions.

    3: 34 suit fit waist - Great jammer! Snug where It's supposed to be, comfortable where you want it to be.
    14: bathing suit suits great -  Toddler bathing suit: comfortable and very good in quality.
    17: shirt great b005qxesww sun -  Comfortable and light weight beach shirt.
    21: shoe water great good -  Speedo Women's Surfwalker Pro: Great Purchase

# Model 7

    Reviews during summer 2014(June - August)

In [39]:
texts = set()

def load7texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            reviewdate = topicdata[areview]['reviewTime']
            datenocomma = reviewdate.replace(",", "")
            mmddyy = datenocomma.split(' ')
            if (int(mmddyy[0])>5 and int(mmddyy[0])<9):
                summary = topicdata[areview]['summary']
                reviewtext = topicdata[areview]['reviewText']
                asin = topicdata[areview]['asin']
                review = '%s %s %s' %(asin,summary,reviewtext)
                texts.add(review)

load7texts(allreviews)

In [40]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: bag b000791api gear mesh
1: beach comfortable pool great
2: small size runs suit
3: cute love great b000c3vb1m
4: four stars b00eymys5s shirt
5: top suit size bottom
6: size big ordered large
7: stars five three great
8: shirt b005qxesww great sun
9: perfect fit great quality
10: great fit product comfortable
11: 34 fit waist size
12: nice well fits swimsuit
13: one would fit time
14: old year loves daughter
15: good quality fit product
16: shoe water great shoes
17: shoes water great fit
18: shorts swim great short
19: feet shoes water pool
20: watch band lap laps
21: suit great bathing fits


In [41]:
import os
outfiles = {}

try:
    os.mkdir('output7')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output7', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()             
        

directory already exists


# Most relevant topic terms and their descriptions.

    1: watch band lap laps -  Watch is waterproof but has ugly band and uneasy to wear.
    6: shirt great sun b005qxesww - Shirt which provides awesome sun protection at the beach.
    17: shoes water great comfortable -  Speedo Women's Surfwalker Pro: Quality water shoes.
    21: 34 suit fit size - Size variations: does not follow the standard range.